# Feature extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/Shareddrives/DM_Project"

Mounted at /content/drive
/content/drive/Shareddrives/DM_Project


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import re as re
import string

from math import e

# Loading the datasets

In [ ]:
tu_df = pd.read_csv("dataset/tweets_users.csv", lineterminator='\n')
user_df = pd.read_csv("dataset/users_clean.csv")

In [ ]:
tu_df.info()
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10230412 entries, 0 to 10230411
Data columns (total 16 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          int64 
 1   name             object
 2   lang             object
 3   bot              int64 
 4   user_timestamp   object
 5   statuses_count   int64 
 6   index            int64 
 7   id               int64 
 8   retweet_count    int64 
 9   reply_count      int64 
 10  favorite_count   int64 
 11  num_hashtags     int64 
 12  num_urls         int64 
 13  num_mentions     int64 
 14  tweet_timestamp  object
 15  text             object
dtypes: int64(11), object(5)
memory usage: 1.2+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11508 non-null  int64 
 1   name            11508 non-null  object
 2   lang            11508 n

In [ ]:
user_df["created_at"] = pd.to_datetime(user_df["created_at"])
user_df.info()
tu_df["text"].fillna("", inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              11508 non-null  int64         
 1   name            11508 non-null  object        
 2   lang            11508 non-null  object        
 3   bot             11508 non-null  int64         
 4   created_at      11508 non-null  datetime64[ns]
 5   statuses_count  11508 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 539.6+ KB


In [ ]:
user_extended = user_df.copy()

# Total number of tweets of each user

In [ ]:
total_tweets = tu_df[["user_id", "id"]].groupby("user_id", as_index=False).count()
total_tweets.rename(columns={"id": "num_tweets", "user_id": "id"}, inplace=True)
user_extended = user_extended.merge(total_tweets)
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296
...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95


# Total of every numeric columns per user # 

In [ ]:
numeric_cols = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']

values_tot = tu_df[["user_id"] + numeric_cols].groupby("user_id", as_index=False).sum()
values_tot.rename(columns={"user_id": "id"}, inplace=True)
for col in numeric_cols:
    values_tot.rename(columns={col: col + "_total"}, inplace=True)

values_tot.info()
user_extended = user_extended.merge(values_tot)
user_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   id                    11508 non-null  int64
 1   retweet_count_total   11508 non-null  int64
 2   reply_count_total     11508 non-null  int64
 3   favorite_count_total  11508 non-null  int64
 4   num_hashtags_total    11508 non-null  int64
 5   num_urls_total        11508 non-null  int64
 6   num_mentions_total    11508 non-null  int64
dtypes: int64(7)
memory usage: 629.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11508 entries, 0 to 11507
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    11508 non-null  int64         
 1   name                  11508 non-null  object        
 2   lang                  11508 non-null  object        
 3   bot    

# Average of every numeric column grouped by user 

In [ ]:
values_avg = tu_df[["user_id"] + numeric_cols].groupby("user_id", as_index=False).mean()
values_avg.rename(columns={"user_id": "id"}, inplace=True)
for col in numeric_cols:
    values_avg.rename(columns={col: col + "_avg"}, inplace=True)

values_avg.info()
user_extended = user_extended.merge(values_avg)
user_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11508 non-null  int64  
 1   retweet_count_avg   11508 non-null  float64
 2   reply_count_avg     11508 non-null  float64
 3   favorite_count_avg  11508 non-null  float64
 4   num_hashtags_avg    11508 non-null  float64
 5   num_urls_avg        11508 non-null  float64
 6   num_mentions_avg    11508 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 629.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11508 entries, 0 to 11507
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    11508 non-null  int64         
 1   name                  11508 non-null  object        
 2   lang                  11508 non-null  object        


# Standard deviation of every numeric column grouped by user #

In [ ]:
# using numpy std() function since it divides for n and not n-1
values_std = tu_df[["user_id"] + numeric_cols].groupby("user_id", as_index=False).std(ddof=0)
values_std.rename(columns={"user_id": "id"}, inplace=True)
for col in numeric_cols:
    values_std.rename(columns={col: col + "_std"}, inplace=True)
values_std.info()
user_extended = user_extended.merge(values_std)
user_extended

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11508 non-null  int64  
 1   retweet_count_std   11508 non-null  float64
 2   reply_count_std     11508 non-null  float64
 3   favorite_count_std  11508 non-null  float64
 4   num_hashtags_std    11508 non-null  float64
 5   num_urls_std        11508 non-null  float64
 6   num_mentions_std    11508 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 629.5 KB


,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,favorite_count_avg,num_hashtags_avg,num_urls_avg,num_mentions_avg,retweet_count_std,reply_count_std,favorite_count_std,num_hashtags_std,num_urls_std,num_mentions_std
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,0.430894,0.154472,0.170732,0.414634,606.286334,0.000000,2.277787,0.477673,0.376274,0.673866
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,0.289720,0.140187,0.158879,0.439252,11582.509571,0.000000,0.947297,0.482373,0.365563,0.629160
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.000000,0.333333,0.666667,0.333333,0.000000,0.000000,0.000000,0.471405,0.471405,0.471405
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,0.476816,0.144513,0.170015,0.525502,12742.155624,0.048094,3.169699,0.592310,0.381768,0.899364
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,1.475121,0.146238,0.143204,0.483010,4408.202034,0.049233,51.897578,0.517741,0.360525,0.779175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,0.555556,0.171627,0.157738,0.464286,11387.340763,0.044499,3.328700,0.608560,0.367207,0.721491
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,0.493631,0.155520,0.157643,0.457537,4150.293447,0.600567,3.457731,0.539045,0.367308,0.785890
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,0.500000,0.000000,0.166667,0.666667,324.975213,0.000000,1.118034,0.000000,0.372678,0.745356
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,0.305263,0.126316,0.126316,0.505263,1127.679785,0.000000,0.871462,0.566468,0.332205,0.806209


# Entropy of every numeric column grouped by user

In [ ]:
def compute_entropy(x):
  vc = x.iloc[:,1].value_counts(normalize=True, sort=False)
  entropy = -(vc * np.log2(vc)).sum()
  return pd.Series({"entropy": entropy})

values_entropy = user_extended[["id"]].copy()

for col in numeric_cols:
  value_entropy = tu_df[["user_id", col]].groupby("user_id", as_index=False).apply(lambda x: compute_entropy(x))
  value_entropy.rename(columns={"user_id": "id", "entropy": f"{col}_entropy"}, inplace=True)
  values_entropy = values_entropy.merge(value_entropy)

user_extended = user_extended.merge(values_entropy)
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,favorite_count_std,num_hashtags_std,num_urls_std,num_mentions_std,retweet_count_entropy,reply_count_entropy,favorite_count_entropy,num_hashtags_entropy,num_urls_entropy,num_mentions_entropy
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,2.277787,0.477673,0.376274,0.673866,1.692384,-0.000000,0.904465,0.635960,0.659376,1.189302
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,0.947297,0.482373,0.365563,0.629160,1.738483,-0.000000,0.862258,0.568979,0.631620,1.217996
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.000000,0.471405,0.471405,0.471405,-0.000000,-0.000000,-0.000000,0.918296,0.918296,0.918296
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,3.169699,0.592310,0.381768,0.899364,2.165086,0.023633,1.042790,0.566554,0.670051,1.375077
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,51.897578,0.517741,0.360525,0.779175,2.252425,0.017488,1.033267,0.598187,0.606409,1.317542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,3.328700,0.608560,0.367207,0.721491,2.205088,0.020672,1.017260,0.651892,0.635140,1.292126
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,3.457731,0.539045,0.367308,0.785890,2.209071,0.028451,1.016673,0.620216,0.635243,1.272963
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,1.118034,0.000000,0.372678,0.745356,0.650022,-0.000000,0.650022,-0.000000,0.650022,1.459148
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,0.871462,0.566468,0.332205,0.806209,1.671046,-0.000000,0.906146,0.335468,0.547247,1.241586


In [ ]:
user_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11508 entries, 0 to 11507
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      11508 non-null  int64         
 1   name                    11508 non-null  object        
 2   lang                    11508 non-null  object        
 3   bot                     11508 non-null  int64         
 4   created_at              11508 non-null  datetime64[ns]
 5   statuses_count          11508 non-null  int64         
 6   num_tweets              11508 non-null  int64         
 7   retweet_count_total     11508 non-null  int64         
 8   reply_count_total       11508 non-null  int64         
 9   favorite_count_total    11508 non-null  int64         
 10  num_hashtags_total      11508 non-null  int64         
 11  num_urls_total          11508 non-null  int64         
 12  num_mentions_total      11508 non-null  int64 

# Tweets per day of ervery user

In [ ]:
reference_day = tu_df["tweet_timestamp"].max()
reference_day

'2020-05-03 10:36:12'

In [ ]:
per_day_tweets = {"id":[], "tweets_per_day":[]}
for row in user_extended[["id", "num_tweets"]].to_numpy().tolist():
  user_id = row[0]
  num_tweets = row[1]
  create_day = user_df[user_df['id']==user_id]['created_at'].iloc[0]
  active_days = (pd.to_datetime(reference_day) - pd.to_datetime(create_day)).days
  per_day_tweets["id"].append(user_id)
  per_day_tweets["tweets_per_day"].append(num_tweets / active_days)


In [ ]:
user_extended = user_extended.merge(pd.DataFrame(per_day_tweets), on="id", how="inner")
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,num_hashtags_std,num_urls_std,num_mentions_std,retweet_count_entropy,reply_count_entropy,favorite_count_entropy,num_hashtags_entropy,num_urls_entropy,num_mentions_entropy,tweets_per_day
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,0.477673,0.376274,0.673866,1.692384,-0.000000,0.904465,0.635960,0.659376,1.189302,0.282759
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,0.482373,0.365563,0.629160,1.738483,-0.000000,0.862258,0.568979,0.631620,1.217996,0.247685
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.471405,0.471405,0.471405,-0.000000,-0.000000,-0.000000,0.918296,0.918296,0.918296,0.001639
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,0.592310,0.381768,0.899364,2.165086,0.023633,1.042790,0.566554,0.670051,1.375077,1.077435
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,0.517741,0.360525,0.779175,2.252425,0.017488,1.033267,0.598187,0.606409,1.317542,10.332288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,0.608560,0.367207,0.721491,2.205088,0.020672,1.017260,0.651892,0.635140,1.292126,6.503226
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,0.539045,0.367308,0.785890,2.209071,0.028451,1.016673,0.620216,0.635243,1.272963,2.556309
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,0.000000,0.372678,0.745356,0.650022,-0.000000,0.650022,-0.000000,0.650022,1.459148,0.003224
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,0.566468,0.332205,0.806209,1.671046,-0.000000,0.906146,0.335468,0.547247,1.241586,0.227818


# Numeric attributes per day

In [ ]:
per_day_stats = {"id": []}
for col in numeric_cols:
  per_day_stats[f"{col}_per_day"] = []

for user_id in user_df["id"].tolist():
  per_day_stats["id"].append(user_id)
  create_day = user_df[user_df['id']==user_id]['created_at'].iloc[0]
  active_days = (pd.to_datetime(reference_day) - pd.to_datetime(create_day)).days
  for col in numeric_cols:
    val = user_extended[user_extended["id"] == user_id][f"{col}_total"].iloc[0]
    per_day_stats[f"{col}_per_day"].append(val / active_days)

In [ ]:
user_extended = user_extended.merge(pd.DataFrame(per_day_stats), on="id", how="inner")
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,num_hashtags_entropy,num_urls_entropy,num_mentions_entropy,tweets_per_day,retweet_count_per_day,reply_count_per_day,favorite_count_per_day,num_hashtags_per_day,num_urls_per_day,num_mentions_per_day
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,0.635960,0.659376,1.189302,0.282759,33.250575,0.000000,0.121839,0.043678,0.048276,0.117241
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,0.568979,0.631620,1.217996,0.247685,370.090278,0.000000,0.071759,0.034722,0.039352,0.108796
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.918296,0.918296,0.918296,0.001639,0.000000,0.000000,0.000000,0.000546,0.001093,0.000546
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,0.566554,0.670051,1.375077,1.077435,847.970858,0.002498,0.513739,0.155704,0.183181,0.566195
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,0.598187,0.606409,1.317542,10.332288,3091.799373,0.018809,15.241379,1.510972,1.479624,4.990596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,0.651892,0.635140,1.292126,6.503226,5167.664516,0.012903,3.612903,1.116129,1.025806,3.019355
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,0.620216,0.635243,1.272963,2.556309,964.131615,0.040706,1.261872,0.397558,0.402985,1.169607
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,-0.000000,0.650022,1.459148,0.003224,0.468565,0.000000,0.001612,0.000000,0.000537,0.002149
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,0.335468,0.547247,1.241586,0.227818,40.824940,0.000000,0.069544,0.028777,0.028777,0.115108


#Average length of the tweets per user#

In [ ]:
def avg_text_length(x):
  avg = 0
  texts = x["text"].to_list()
  for text in texts:
    avg += len(text)
  return avg/len(texts)


avg_len = tu_df.groupby("user_id",  as_index=False).apply(lambda x: avg_text_length(x))
avg_len.rename(columns={"user_id": "id", None :"avg_len"}, inplace=True)
user_extended = user_extended.merge(avg_len)
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,num_urls_entropy,num_mentions_entropy,tweets_per_day,retweet_count_per_day,reply_count_per_day,favorite_count_per_day,num_hashtags_per_day,num_urls_per_day,num_mentions_per_day,avg_len
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,0.659376,1.189302,0.282759,33.250575,0.000000,0.121839,0.043678,0.048276,0.117241,60.845528
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,0.631620,1.217996,0.247685,370.090278,0.000000,0.071759,0.034722,0.039352,0.108796,68.000000
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.918296,0.918296,0.001639,0.000000,0.000000,0.000000,0.000546,0.001093,0.000546,17.333333
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,0.670051,1.375077,1.077435,847.970858,0.002498,0.513739,0.155704,0.183181,0.566195,86.453632
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,0.606409,1.317542,10.332288,3091.799373,0.018809,15.241379,1.510972,1.479624,4.990596,71.771845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,0.635140,1.292126,6.503226,5167.664516,0.012903,3.612903,1.116129,1.025806,3.019355,51.500992
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,0.635243,1.272963,2.556309,964.131615,0.040706,1.261872,0.397558,0.402985,1.169607,58.939490
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,0.650022,1.459148,0.003224,0.468565,0.000000,0.001612,0.000000,0.000537,0.002149,67.666667
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,0.547247,1.241586,0.227818,40.824940,0.000000,0.069544,0.028777,0.028777,0.115108,62.568421


#Average number of special characters in the tweets per user#

In [ ]:
def avg_special_chars(x):
  avg = 0
  texts = x["text"].to_list()
  for text in texts:
    avg += len(re.findall("[^a-z^A-Z^0-9]", text.replace(" ", "")))
  return avg/len(texts)

avg_specials = tu_df.groupby("user_id",  as_index=False).apply(lambda x: avg_special_chars(x))
avg_specials.rename(columns={"user_id": "id", None:"avg_spc_len"}, inplace=True)

In [ ]:
user_extended = user_extended.merge(avg_specials)
user_extended

,id,name,lang,bot,created_at,statuses_count,num_tweets,retweet_count_total,reply_count_total,favorite_count_total,...,num_mentions_entropy,tweets_per_day,retweet_count_per_day,reply_count_per_day,favorite_count_per_day,num_hashtags_per_day,num_urls_per_day,num_mentions_per_day,avg_len,avg_spc_len
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,123,14464,0,53,...,1.189302,0.282759,33.250575,0.000000,0.121839,0.043678,0.048276,0.117241,60.845528,3.910569
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,107,159879,0,31,...,1.217996,0.247685,370.090278,0.000000,0.071759,0.034722,0.039352,0.108796,68.000000,3.560748
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,3,0,0,0,...,0.918296,0.001639,0.000000,0.000000,0.000000,0.000546,0.001093,0.000546,17.333333,4.333333
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,1294,1018413,3,617,...,1.375077,1.077435,847.970858,0.002498,0.513739,0.155704,0.183181,0.566195,86.453632,5.537094
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,3296,986284,6,4862,...,1.317542,10.332288,3091.799373,0.018809,15.241379,1.510972,1.479624,4.990596,71.771845,3.571602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1008,800988,2,560,...,1.292126,6.503226,5167.664516,0.012903,3.612903,1.116129,1.025806,3.019355,51.500992,4.896825
11504,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1884,710565,30,930,...,1.272963,2.556309,964.131615,0.040706,1.261872,0.397558,0.402985,1.169607,58.939490,4.214968
11505,126984069,ALMA LETICIA NUÑO,es,0,2015-03-29 17:01:24,6,6,872,0,3,...,1.459148,0.003224,0.468565,0.000000,0.001612,0.000000,0.000537,0.002149,67.666667,6.666667
11506,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,95,17024,0,29,...,1.241586,0.227818,40.824940,0.000000,0.069544,0.028777,0.028777,0.115108,62.568421,3.926316


# Save the new dataset

In [ ]:
user_extended.to_csv("./dataset/users_extended.csv", index=False)

In [ ]:
temp = pd.read_csv("./dataset/users_extended.csv")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      11508 non-null  int64  
 1   name                    11508 non-null  object 
 2   lang                    11508 non-null  object 
 3   bot                     11508 non-null  int64  
 4   created_at              11508 non-null  object 
 5   statuses_count          11508 non-null  int64  
 6   num_tweets              11508 non-null  int64  
 7   retweet_count_total     11508 non-null  int64  
 8   reply_count_total       11508 non-null  int64  
 9   favorite_count_total    11508 non-null  int64  
 10  num_hashtags_total      11508 non-null  int64  
 11  num_urls_total          11508 non-null  int64  
 12  num_mentions_total      11508 non-null  int64  
 13  retweet_count_avg       11508 non-null  float64
 14  reply_count_avg         11508 non-null